# Experimental Setup for Building Classifiers:

## Egyptian Wikipedia Dataset Encoding Using Spark-NLP:

### Encoding Procedure Using "[Word2Vec Embeddings in Egyptian Arabic (300d)](https://sparknlp.org/2022/03/14/w2v_cc_300d_arz_3_0.html)":

In [3]:
import os
import sparknlp
import numpy as np
import pandas as pd
spark = sparknlp.start()
from sparknlp.base import *
from sparknlp.base import LightPipeline
from IPython.display import clear_output
from nltk.tokenize import WhitespaceTokenizer
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (Tokenizer, WordEmbeddingsModel)

documentAssembler = DocumentAssembler().setInputCol('text').setOutputCol('document')
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d","arz").setInputCols(["document", "token"]).setOutputCol("embeddings")
pipeline = Pipeline().setStages([documentAssembler, tokenizer, embeddings])
data = spark.createDataFrame([['']]).toDF("text")
arz_model = pipeline.fit(data)

clear_output(wait=True)
print("", flush=True)

In [4]:
articles = 0
pages_embeddings = {}

def get_sparknlp_arz_embeddings(word, model):
    light_model_emb = LightPipeline(pipelineModel=arz_model, parse_embeddings=True)
    annotate_results_emb = light_model_emb.annotate(word)
    strVectors = annotate_results_emb['embeddings'][0]
    
    with open('word.txt', 'w') as f: f.write(str(strVectors))
    f = open('word.txt', 'r'); vectors = f.read(); f.close()
    os.remove("word.txt") 
    
    lvectors = vectors.replace("'","")
    lvectors = vectors.replace("E","e")
    listVectors = lvectors.split(" ")
    
    word2vector = []
    for s in listVectors: word2vector.append(float(s))
    return word2vector

def encode_page_text(page_text, dataframe_name): 
    counter = 0
    global articles
    articles += 1
    vectors = [0] * 300
    
    tokenizer = WhitespaceTokenizer()
    words = tokenizer.tokenize(page_text)

    idx = np.where(dataframe_name["page_text_cleaned"] == page_text)[0][0]
    page_title = dataframe_name['page_title'][idx]

    for word in words:
        try:
            wv = get_sparknlp_arz_embeddings(word, arz_model)
            vectors = [x + y for x, y in zip(vectors, wv)]
            counter += 1
                
        except: continue
                
    clear_output(wait=True)
    print(f"# Texts of {articles} pages have been encoded.", flush=True)

    encoded_page_text = [x / counter for x in vectors]
    
    pages_embeddings[page_title] = encoded_page_text
    
    return encoded_page_text

In [2]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

df = pd.read_csv('arzwiki-20240101-all-20k.csv')
df.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated


In [5]:
df['page_text_encoded_sparknlp'] = df['page_text_cleaned'].apply(lambda page_text: encode_page_text(page_text, df))
df.head()

# Texts of 20000 pages have been encoded.


,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label,page_text_encoded_sparknlp
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated,"[-0.01276949152542373, 0.00048644067796609894,..."
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated,"[-0.0033208955223880602, -0.04084029850746269,..."
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated,"[0.004411607142857147, -0.025443750000000008, ..."
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated,"[0.0031707964601769916, -0.0018451327433628292..."
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated,"[0.014313826679649486, -0.013468354430379716, ..."


### Exporting and Importing Word Embeddings:

In [6]:
import json

with open('arzwiki-20240101-all-20k-sparknlp-embeddings.json', 'w') as json_file:
    json.dump(pages_embeddings, json_file)

In [7]:
import json
 
with open('arzwiki-20240101-all-20k-sparknlp-embeddings.json') as json_file:
    fetched_pages_embeddings = json.load(json_file)

In [8]:
import numpy as np

def fetch_embeddings(page_text, dataframe_name, embeddings_file):
    idx = np.where(dataframe_name["page_text_cleaned"] == page_text)[0][0]
    page_title = dataframe_name['page_title'][idx]
    return embeddings_file[page_title]

In [9]:
df['fetched_page_text_sparknlp_embeddings'] = df['page_text_cleaned'].apply(lambda page_text: fetch_embeddings(page_text, df, fetched_pages_embeddings))
df.head()

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label,page_text_encoded_sparknlp,fetched_page_text_sparknlp_embeddings
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated,"[-0.01276949152542373, 0.00048644067796609894,...","[-0.01276949152542373, 0.00048644067796609894,..."
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated,"[-0.0033208955223880602, -0.04084029850746269,...","[-0.0033208955223880602, -0.04084029850746269,..."
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated,"[0.004411607142857147, -0.025443750000000008, ...","[0.004411607142857147, -0.025443750000000008, ..."
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated,"[0.0031707964601769916, -0.0018451327433628292...","[0.0031707964601769916, -0.0018451327433628292..."
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated,"[0.014313826679649486, -0.013468354430379716, ...","[0.014313826679649486, -0.013468354430379716, ..."


24/02/06 18:18:24 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/_4/w9tlcqr16zv0gnfg0msmj9x00000gn/T/blockmgr-89c57090-5674-452e-9107-71cc918b1b5c. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/_4/w9tlcqr16zv0gnfg0msmj9x00000gn/T/blockmgr-89c57090-5674-452e-9107-71cc918b1b5c
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1141)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:182)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:178)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.Indexe